In [56]:
import numpy as np
import pickle
import time
import itertools
from operator import add
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [68]:
# board = [[1x1 pieces], [1x2 pieces], [2x1 pieces], [2x2 pieces]] (the lower-left block of each piece)
start_board_state = [[[0, 0]], [[1, 1]], [[2, 1]], [[3, 0]], 
                     [[0, 3], [0, 4]], [[0, 1], [0, 2]], [[3, 3], [3, 4]], [[3, 1], [3, 2]], 
                     [[1, 2], [2, 2]], 
                     [[1, 3], [1, 4], [2, 3], [2, 4]]]

start_board_state = [[np.array(block) for block in piece] for piece in start_board_state]

In [72]:
start_board_state

[[array([1, 0])],
 [array([1, 1])],
 [array([2, 1])],
 [array([3, 0])],
 [array([0, 3]), array([0, 4])],
 [array([0, 1]), array([0, 2])],
 [array([3, 3]), array([3, 4])],
 [array([3, 1]), array([3, 2])],
 [array([1, 2]), array([2, 2])],
 [array([1, 3]), array([1, 4]), array([2, 3]), array([2, 4])]]

In [48]:
class Node:
    def __init__(self, board_state):
        self.parent = None
        self.board_state = board_state
        self.children = []
        self.add_to_history(self.compress(self.board_state))
    
    def add_to_history(self, compressed_board):    
        global history
        history.add(compressed_board)      
        return
    
    def compress(self, board_state):
        return str(sorted([str(piece) for piece in start_board_state]))
    
    def  generate_children(self):
        global history
        for piece_type in range(len(self.board_state)):
            for piece in range(len(self.board_state[piece_type])):
                for direction in np.array([[-1, 0], [0, -1], [0, 1], [1, 0]]):
                    potential_child = self.move_if_possible(piece_type, piece, direction)
                    if potential_child == None:
                        pass
                    elif self.compress(potential_child) not in history:
                        self.children.append(Node(potential_child))
        return
    
    def move_if_possible(self, piece_type, piece, direction):
        
        
        
        
        board_without_piece = [[list(piece) for piece in piece_type] for piece_type in self.board_state]
        del board_without_piece[piece_type][piece]
        moved_piece = [sum(x) for x in zip(self.board_state[piece_type][piece][:], direction)]
        expanded_piece = self.expand_piece(piece_type, moved_piece)
        for block in expanded_piece:
            if block[0] < 0 or block[0] > 3 or block[1] < 0 or block[1] > 4:
                return
            for piece_type_wo in board_without_piece:
                for piece_wo in piece_type_wo:
                    if block == piece_wo:
                        return
        all_blocks_wo = []
        upper_right_expanded = max(expanded_piece)
        for piece_type_wo in range(len(board_without_piece)):
            for piece_wo in board_without_piece[piece_type_wo]:
                if piece_wo[0] <= upper_right_expanded[0] or piece_wo[1] <= upper_right_expanded[1]:
                    expanded_wo = self.expand_piece(piece_type_wo, piece_wo)
                    for block in expanded_wo:
                        if block in expanded_piece:
                            return
        output = [[piece[:] for piece in piece_type] for piece_type in self.board_state]
        output[piece_type][piece] = moved_piece
        return output
    
    def expand_piece(self, piece_type, piece):
        if piece_type == 0:
            output = [piece]
        elif piece_type == 1:
            output = [piece, list(map(add, piece, [0, 1]))]
        elif piece_type == 2:
            output = [piece, list(map(add, piece, [1, 0]))]
        elif piece_type == 3:
            output = [piece, list(map(add, piece, [1, 0])), 
                      list(map(add, piece, [0, 1])), 
                      list(map(add, piece, [1, 1]))]
        return output

    def draw_board(self):
        expanded_board = []
        for piece_type in range(len(self.board_state)):
            for piece in self.board_state[piece_type]:
                expanded_board.append(self.expand_piece(piece_type, piece))
        board = np.full((5, 4), ' ')
        for i in range(10):
            # X-Wert
            for x in range(len(expanded_board[i])):
                board[expanded_board[i][x][1]][expanded_board[i][x][0]] = i
        return np.flip(np.fliplr(board))

In [28]:
def parent_pointer(child, parent):
    child.parent = parent

In [29]:
def get_layer(root, depth):
    root = [root]
    for layer in range(depth):
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
            root = new_root
    return root

In [30]:
def get_last_layer(root):
    root = [root]
    depth = 0
    while True:
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
        if new_root == []:
            return root
        root = new_root
        depth += 1

In [31]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.history = []

In [32]:
def save(root):
    with open(file, 'wb') as f:
        pickle.dump(root, f, pickle.HIGHEST_PROTOCOL)
        
def load(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [50]:
# %%prun
history = {*()}

root = Node(start_board_state)
i = 0
layer = get_layer(root, 0)
while not layer == []:
    [node.generate_children() for node in layer]
    for node in layer:
        for child in node.children:
            parent_pointer(node, child)
    print('layer:', i, 'nodes:', len(layer))
    i += 1
    layer = get_layer(root, i)

UnboundLocalError: local variable 'output' referenced before assignment

In [10]:
i = 0
total_nodes = 0
while True:
    layer = get_layer(root, i)
    if layer == []:
        break
    total_nodes += len(layer)
    i += 1
print('total number of possible board states: %s'% total_nodes)

total number of possible board states: 25955


https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

https://towardsdatascience.com/speed-up-jupyter-notebooks-20716cbe2025

In [26]:
start_board_state

[[array([0, 0])],
 [array([1, 1])],
 [array([2, 1])],
 [array([3, 0])],
 [array([0, 3]), array([0, 4])],
 [array([0, 1]), array([0, 2])],
 [array([3, 3]), array([3, 4])],
 [array([3, 1]), array([3, 2])],
 [array([1, 2]), array([2, 2])],
 [array([1, 3]), array([1, 4]), array([2, 3]), array([2, 4])]]